In [ ]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import ccf
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.stattools import acf, pacf
from scipy.stats import norm
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

load_dotenv()

In [ ]:
# ...existing code...
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import numpy as np
import pandas as pd

# Opção 1: ignorar todos os warnings (global)
warnings.filterwarnings("ignore")

# Opção 2: ignorar apenas categorias específicas (recomendado)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Pandas: desativar aviso de SettingWithCopy (use com cuidado)
pd.options.mode.chained_assignment = None

# NumPy: ignorar avisos numéricos (overflow/underflow etc.)
np.seterr(all="ignore")
# ...existing code...

# Importando os dados

Dados previamente tratados.
Foram importados 48 índices

In [ ]:
# Carregar variáveis de ambiente
load_dotenv()

path_dados_tratados = os.getenv('PATH_DADOS_TRATADOS')

dados_indicadores = pd.read_csv(path_dados_tratados + "dados_indicadores_tratados.csv")
dados_indices = pd.read_csv(path_dados_tratados + "dados_indices.csv", sep=';')
dados_indices.drop(columns=['Unnamed: 0'], inplace=True)
dados_indices.drop(columns=['incc'], inplace=True)
preco_acoes = pd.read_csv(path_dados_tratados + 'preco_acoes.csv')
preco_acoes = preco_acoes.iloc[:-1, :]

In [ ]:
dados_indices.info()

Renomear colunas

In [ ]:
rename_map = {
    "IPAOG-DI Ipa - todos os itens - Nro. Índice(1477007)": "IPAOG-DI",
    "INCC-Brasil-DI-Todos os itens(1464783)": "INCC-Brasil-DI",
    "Indicador de Incerteza da Economia Brasil (IIE-Br)(1428452)": "IIE-Br",
    "Indicador de Incerteza da Economia Brasil - Componente Mídia (Mídia - IIE-Br)(1463188)": "Mídia - IIE-Br",
    "Indicador de Incerteza da Economia Brasil - Componente Expactativas (Expec IIE-Br)(1463189)": "Expec IIE-Br",
    "Indicador Antecedente de Emprego (IA Emp) com ajuste sazonal(1416205)": "IA Emp",
    "Indicador Antecedente de Emprego (IA Emp) com ajuste sazonal(1416205) (Variação Percentual em Relação ao Período Anterior)": "Var. IA Emp",
    "PIB a preços de 1995(1428699)": "PIB 1995",
    "PIB a preços correntes(1428698)": "PIB Corrente",
    "Índice de volume mensal  - PIB a preços de mercado (1428676)": "Vol. PIB Mensal",
    "Índice de volume mensal pela ótica da produção - Valor adicionado a preços básicos(1428677)": "Vol. VA Produção",
    "Índice de volume mensal pela ótica da produção - Valor adicionado a preços básicos - Agropecuária(1428678)": "Vol. VA Agro",
    "Índice de volume mensal pela ótica da produção - Valor adicionado a preços básicos - Indústria(1428679)": "Vol. VA Indústria",
    "Índice de volume mensal pela ótica da produção - Valor adicionado a preços básicos - Serviços(1428680)": "Vol. VA Serviços",
    "Índice de volume mensal pela ótica da produção - Impostos (1428681)": "Vol. Impostos",
    "Índice de volume mensal com ajuste sazonal - PIB a preços de mercado (1428682)": "Vol. PIB Sazonal",
    "Índice de volume mensal pela ótica da produção com ajuste sazonal -Valor adicionado (1428683)": "Vol. VA Sazonal",
    "Índice de volume mensal pela ótica da produção com ajuste sazonal - Valor adicionado - Agropecuária(1428684)": "Vol. VA Agro Sazonal",
    "Índice de volume mensal pela ótica da produção com ajuste sazonal - Valor adicionado - Indústria(1428685)": "Vol. VA Ind. Sazonal",
    "Índice de volume mensal pela ótica da produção com ajuste sazonal - Valor adicionado - Serviços(1428686)": "Vol. VA Serv. Sazonal",
    "Índice de volume trimestral com ajuste sazonal - PIB(1428700)": "Vol. PIB Trim. Saz.",
    "Índice de volume trimestral com ajuste sazonal - Valor Adicionado(1428701)": "Vol. VA Trim. Saz.",
    "Índice de volume trimestral com ajuste sazonal - Valor Adicionado - Agropecuária(1428702)": "Vol. VA Agro Trim. Saz.",
    "Índice de volume trimestral com ajuste sazonal - Valor Adicionado - Indústria(1428703)": "Vol. VA Ind. Trim. Saz.",
    "Índice de volume trimestral com ajuste sazonal - Valor Adicionado - Serviços(1428704)": "Vol. VA Serv. Trim. Saz.",
    "Índice da Taxa de Câmbio Real - (Real / Dólar (EUA))(1003592)": "Câmbio Real/Dólar",
    "Índice da Taxa de Câmbio Real - (Real / Iene (Japão))(1003593)": "Câmbio Real/Iene",
    "Índice da Taxa de Câmbio Real - (Real / Peso (Argentina))(1003594)": "Câmbio Real/Peso",
    "Índice da Taxa de Câmbio Real - (Real / Euro (Zona do Euro))(1003595)": "Câmbio Real/Euro",
    "Índice da Taxa de Câmbio Real - (Real / Libra (Reino Unido))(1003596)": "Câmbio Real/Libra",
    "Índice da Taxa de Câmbio Efetiva Real - (Real / Cesta de Moedas)(1003597)": "Câmbio Efetiva Real",
    "Juros EUA": "Juros EUA",
    "Tbonds": "T-bonds",
    "Cotacao dolar": "Cotação Dólar",
    "CDI": "CDI",
    "TJLP": "TJLP",
    "Energia res": "Energia Res.",
    "Energia ind": "Energia Ind.",
    "Energia comer": "Energia Com.",
    "Divida/PIB": "Dívida/PIB",
}

if 'dados_indices' in globals():
    dados_indices.rename(columns=rename_map, inplace=True)

# Opcional: verificar quais colunas foram renomeadas em cada DF
for name, df in [('dados_indicadores', globals().get('dados_indicadores')),
                 ('dados_indices', globals().get('dados_indices'))]:
    if df is not None:
        renomeadas = [v for k, v in rename_map.items() if k in df.columns]
        print(f"{name}: {len(renomeadas)} colunas renomeadas.")
# ...existing code...

In [ ]:
dados_indices.info()

Como o foco deste estudo são as companhias de transmissão de energia elétrica, as seguintes empresas serão consideradas:
* ELET6 - CD_CVM = 2437
* ISAE4 - CD_CVM = 18376
* TAEE4 - CD_CVM = 20257
* ALUP4 - CD_CVM = 21490

In [ ]:
# Filtrar dataframe para possuir apenas os códigos 2437, 18376, 20257, 21490
dados_indicadores_transm = dados_indicadores[dados_indicadores['CD_CVM'].isin([2437, 18376, 20257, 21490])]
dados_indicadores_transm.query("CD_CVM == 21490").head()

Interpolando valores ausentes

In [ ]:
novas_linhas = pd.DataFrame({
    'CD_CVM': [21490, 21490],
    'DENOM_CIA' : ['ALUPAR INVESTIMENTO S/A', 'ALUPAR INVESTIMENTO S/A'],
    'DT_FIM_EXERC': ['2010-06-30', '2011-06-30'],
    '3.01': [np.nan, np.nan],
    '3.02': [np.nan, np.nan],
    '3.03': [np.nan, np.nan],
    '3.04': [np.nan, np.nan],
    '3.05': [np.nan, np.nan],
    '3.06': [np.nan, np.nan],
    '3.07': [np.nan, np.nan],
    '3.08': [np.nan, np.nan],
    '3.09': [np.nan, np.nan],
    '3.11': [np.nan, np.nan]
})

In [ ]:
dados_indicadores_transm = pd.concat([dados_indicadores_transm, novas_linhas], ignore_index=True)

In [ ]:
#interpolar os valores nulos
dados_indicadores_transm = dados_indicadores_transm.sort_values(by=['CD_CVM', 'DT_FIM_EXERC'])
dados_indicadores_transm = dados_indicadores_transm.interpolate(method='linear', limit_direction='both')

Organiza indicadores

In [ ]:
dict_indicadores = {}
for cd_cvm in dados_indicadores_transm['CD_CVM'].unique():
    df_temp = dados_indicadores_transm.copy()
    df_temp = df_temp[df_temp['CD_CVM'] == cd_cvm].copy()
    
    df_temp.drop(columns=['CD_CVM','DENOM_CIA'], inplace=True)
    df_temp.set_index('DT_FIM_EXERC', inplace=True)
    dict_indicadores[cd_cvm] = df_temp



In [ ]:
# # Interpolar NaNs em dict_indicadores_estac_norm[empresa] para cada empresa
# for emp, df in dict_indicadores.items():
#     before = int(df.isna().sum().sum())
#     df2 = df.copy()

#     # Garantir índice datetime para interpolação por tempo
#     try:
#         new_index = pd.to_datetime(df2.index, errors='raise')
#         df2 = df2.set_index(new_index).sort_index()
#         df2 = df2.interpolate(method='time', limit_direction='both', axis=0)
#     except Exception:
#         # Fallback para linear se índice não for datetime
#         df2 = df2.sort_index().interpolate(method='linear', limit_direction='both', axis=0)

#     # Opcional: se sobrar algum NaN, usar ffill/bfill nas bordas
#     if df2.isna().any().any():
#         df2 = df2.ffill().bfill()

#     dict_indicadores[emp] = df2
#     after = int(df2.isna().sum().sum())
#     print(f"{emp}: NaNs {before} -> {after}")

# # Conferir uma empresa ativa
# dict_indicadores[empresa]
# # ...existing code...

Normalização

In [ ]:
def normalize_dataframe(df):
    """
    Z-score normalization para todas as colunas de um DataFrame.
    Retorna um novo DataFrame cujas colunas são (x - mean) / std.
    """
    return (df - df.mean()) / (df.max() - df.min())

Tornando variáveis exógenas estacionárias
* Normalização é aplicada depois de tornar o sinal estacionário no caso das variáveis exógenas
* Para as variáveis dependentes, basta normalizá-las, umas vez que a parte integrativa dos modelos irá cuidar da estacionariedade. Neste caso é necessário saber apenas o grau de diferenciação que precisa ser realizada

In [ ]:
def tornar_estacionaria(serie, max_diff=5, verbose=False):
    """
    Aplica diferenciações sucessivas até a série temporal ficar estacionária (ADF p < 0.05).
    Retorna a série estacionária e o número de diferenciações aplicadas.
    """
    serie_estac = serie.copy()
    diff_count = 0
    for _ in range(max_diff + 1):
        adf_result = adfuller(serie_estac.dropna())
        p_value = adf_result[1]
        if verbose:
            print(f"Differencing: {diff_count}, p-value: {p_value:.4f}")
        if p_value < 0.05:
            return serie_estac, diff_count
        serie_estac = serie_estac.diff().dropna()
        diff_count += 1
    if verbose:
        print("Atenção: não ficou estacionária após", max_diff, "diferenciações.")
    return serie_estac, diff_count

In [ ]:
indices_estac = dados_indices.copy()
for column in dados_indices.columns:
    if column != 'data':
        indices_estac[column], diffs = tornar_estacionaria(dados_indices[column], verbose=True)
        print(f"Série '{column}' tornou-se estacionária após {diffs} diferenciações.")
indices_estac.set_index(dados_indices['data'], inplace=True)
indices_estac = indices_estac.iloc[1:, :]

In [ ]:
indicadores_estac = {}
for cd_cvm, df in dict_indicadores.items():
    indicadores_estac[cd_cvm] = pd.DataFrame()
    for column in df.columns:
        indicadores_estac[cd_cvm][column], diffs = tornar_estacionaria(df[column], verbose=True)
        print(f"CD_CVM {cd_cvm} - Série '{column}' tornou-se estacionária após {diffs} diferenciações.")
    print(len(indicadores_estac[cd_cvm]))
    if len(indicadores_estac[cd_cvm]) == 58:
        indicadores_estac[cd_cvm].set_index(df.index[1:], inplace=True)
    else:
        indicadores_estac[cd_cvm] = indicadores_estac[cd_cvm].iloc[1:, :]
        indicadores_estac[cd_cvm].set_index(df.index[1:], inplace=True)

Verificar sazonalidade

In [ ]:
# ...existing code...
def plot_com_destaques(serie, n=4, offset=0, titulo=None, label='Série'):
    s = pd.Series(serie).reset_index(drop=True)
    x = np.arange(len(s))

    plt.figure(figsize=(10, 5))
    plt.plot(x, s.values, label=label, color='steelblue', linewidth=1.5)

    idx = np.arange(offset, len(s), n)  # ex.: 0,4,8,... (use offset=1 para 1,5,9,...)
    plt.scatter(idx, s.iloc[idx].values, color='crimson', s=60, zorder=3,
                edgecolors='white', linewidths=0.6, label=f'Destaque terceiro trimestre')

    plt.title(titulo or f'{label} (Destaque terceiro trimestre)')
    plt.xlabel('Posição')
    plt.ylabel('Valor')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Uso com a sua série:
for code in [2437, 18376, 20257, 21490]:
    plot_com_destaques(dados_indicadores.query(f"CD_CVM == {code}")['3.11'], n=4, offset=2, titulo=f'Empresa {code} - Lucro', label='3.01')

In [ ]:
plot_acf(dados_indicadores.query(f"CD_CVM == 20257")['3.01'])

Correlação Cruzada

O foco de predição será nos indicadores:
* 3.01 (Receitas de venda de Bens e/ou Serviços)
* 3.02 (Custo dos Bens e/ou Serviços)
* 3.11 (Lucro ou Prejuízo consolidado do Período)

In [ ]:
# ...existing code...
def plot_ccf(lags, corr, indice, col, empresa=None):
    """
    Plota a CCF com linhas de confiança, eixos e legendas padronizados.
    - lags: array de lags
    - corr: array de correlações (mesmo tamanho de lags)
    - indice: nome da variável X
    - indicador: nome da variável Y
    - empresa: identificador opcional
    - n_obs: nº de observações para IC; se None usa len(corr) como aproximação
    - alpha: nível de significância (default 5%)
    """
    fig, ax =plt.subplots(figsize=(15, 6))
    plt.stem(lags, corr, use_line_collection=True)
    plt.title(f'CCF entre {indice} e {col} - Empresa {empresa}')
    confidence_interval = 1.96/np.sqrt(len(corr))
    ax.axhline(-confidence_interval, color='red', label='5% intervalo de confiança')
    ax.axhline(confidence_interval, color='red')
    ax.axvline(x = 0, color = 'black', lw = 1)
    ax.axhline(y = 0, color = 'black', lw = 1)
    ax.axhline(y = np.max(corr), color = 'blue', lw = 1,
            linestyle='--', label = 'Maior correlação +/-')
    ax.axhline(y = np.min(corr), color = 'blue', lw = 1,
            linestyle='--')
    ax.set(ylim = [-1, 1])
    # if y_label != x_label:
    #     ax.set_title('Correlação Cruzada ' + x_label + ' e ' + y_label, weight='bold', fontsize = 15)
    # if y_label == x_label:
    #     ax.set_title('Autocorrelação ' + x_label, weight='bold', fontsize = 15)
    ax.set_ylabel('Coeficientes de Correlação', weight='bold',
    fontsize = 12)
    #determina os pontos marcados no eixo X
    plt.xticks(range(min(lags), max(lags+1), 3))
    ax.set_xlabel('Lags', weight='bold', fontsize = 12)
    plt.legend()
# ...existing code...

In [ ]:
# Calcular a Função de Correlação Cruzada (CCF) entre índices (VIF) e apenas os indicadores alvo
ccf_dict = {}
alvo_indicadores = ['3.01', '3.02', '3.11']

# Número de observações para o cálculo do intervalo de confiança (aprox.)
n_obs = len(indicadores_estac[2437])
conf_interval = 1.96 / np.sqrt(n_obs)

for empresa, df in indicadores_estac.items():
    # Indicadores alvo disponíveis neste df
    cols_indicadores = [c for c in alvo_indicadores if c in df.columns]
    if not cols_indicadores:
        continue

    # # Índices aprovados pelo VIF que existem em indices_norm_estac
    # cols_indices = [c for c in vif_finais.get(empresa, pd.Series(dtype=float)).index
    #                 if c in indices_norm_estac.columns]
    # if not cols_indices:
    #     continue

for indice in indices_estac.columns[1:]:
    serie_x = indices_estac[indice].dropna()
    for empresa, df in indicadores_estac.items():
        for col in cols_indicadores:
            serie_y = df[col].dropna()

            # Alinhar pelas datas em comum
            #print(f'{serie_x.shape} - {serie_y.shape}')

            aligned = pd.concat([serie_x, serie_y], axis=1, join='inner').dropna()
            if len(aligned) < 3:
                continue

            corr = ccf(aligned.iloc[:, 0], aligned.iloc[:, 1], adjusted=False)
            lags = np.arange(len(corr))
            ccf_dict[(empresa, indice, col)] = corr

In [ ]:
ccf_dict[(2437, 'ibov', '3.01')]

In [ ]:
# Desempacotar ccf_dict em um DataFrame
records = []
for (empresa, indice_x, indicador), ccf_arr in ccf_dict.items():
    for lag, corr_val in enumerate(ccf_arr):
        records.append({
            'Indice': indice_x,
            'Empresa': empresa,
            'Indicador': indicador,
            'Lag': lag,          # statsmodels.ccf retorna lags >= 0
            'CCF': corr_val,
            'Abs_CCF': abs(corr_val)
        })

df_ccf = pd.DataFrame(records)

# Opcional: limitar por lag máximo (ex.: até 8)
lag_max = 8
df_ccf = df_ccf[df_ccf['Lag'] <= lag_max]

# Marcar significância pelo intervalo de confiança já calculado (conf_interval)
df_ccf['Significativo'] = df_ccf['Abs_CCF'] > conf_interval

# 1) Top N lags significativos em geral
topN = 10
top_lags = (df_ccf[df_ccf['Significativo']]
            .sort_values('Abs_CCF', ascending=False)
            .head(topN))
display(top_lags)

# 2) Ranking por par (Indice, Empresa, Indicador): melhor lag por par
idx_max = (df_ccf[df_ccf['Significativo']]
           .groupby(['Empresa', 'Indice', 'Indicador'])['Abs_CCF']
           .idxmax())
ranked_pairs = (df_ccf.loc[idx_max]
                .sort_values('Abs_CCF', ascending=False)
                .reset_index(drop=True))
display(ranked_pairs.head(topN))

In [ ]:
# Plotar Top 5 por (Empresa, Indicador) com base em ranked_pairs
top_n = 5
alvo_indicadores = ['3.01', '3.02', '3.11']  # opcional: filtre apenas os indicadores-alvo

rp = ranked_pairs.copy()
if 'Indicador' in rp.columns and alvo_indicadores:
    rp = rp[rp['Indicador'].isin(alvo_indicadores)]

for (empresa, indicador), df_group in rp.groupby(['Empresa', 'Indicador']):
    df_top = df_group.sort_values('Abs_CCF', ascending=False).head(top_n)
    for _, row in df_top.iterrows():
        indice = row['Indice']
        key = (empresa, indice, indicador)
        corr = ccf_dict.get(key)
        if corr is None:
            # chave inexistente no dicionário (pode ocorrer por nomes/filtragens)
            continue
        lags = np.arange(len(corr))
        plot_ccf(lags, corr, indice=indice, col=indicador, empresa=empresa)

In [ ]:
dados_indicadores_transm.info()

Montando dataframes de indicadores específicos

In [ ]:
def criar_dataframe_indicador(df, coluna_valor):
    """
    Cria um dataframe pivotado onde a coluna 'DT_FIM_EXERC' é o índice
    e cada coluna corresponde ao valor da coluna especificada (coluna_valor)
    de cada empresa (CD_CVM).

    Parâmetros:
      df: DataFrame original com as colunas 'CD_CVM', 'DT_FIM_EXERC' e a coluna desejada.
      coluna_valor: Nome da coluna que será utilizada para os valores (ex.: 'Resultado Bruto').

    Retorna:
      Um DataFrame pivotado.
    """
    resultado = pd.DataFrame()
    for cd_cvm in df['CD_CVM'].unique():
        df_temp = df.loc[df['CD_CVM'] == cd_cvm, ['DT_FIM_EXERC', coluna_valor]].copy()
        df_temp = df_temp.rename(columns={coluna_valor: cd_cvm})
        df_temp = df_temp.set_index('DT_FIM_EXERC')
        if resultado.empty:
            resultado = df_temp.copy()
        else:
            resultado = resultado.join(df_temp, how='outer')

    # Preencher valores NaN com 0
    resultado.fillna(0, inplace=True)
    
    return resultado

In [ ]:
resultado_bruto = criar_dataframe_indicador(dados_indicadores_transm, '3.01')
lucro_consolidado = criar_dataframe_indicador(dados_indicadores_transm, '3.11')
despesas_operacionais = criar_dataframe_indicador(dados_indicadores_transm, '3.04')

In [ ]:
dados_indicadores_transm['CD_CVM'].unique()

Teste de estacionariedade KPSS
-   **Hipótese Nula (H0)**: A série é estacionária.
-   **Hipótese Alternativa (Ha)**: A série não é estacionária.

Teste de estacionariedade ADF
-   **Hipótese Nula (H0)**: A série não é estacionária.
-   **Hipótese Alternativa (Ha)**: A série é estacionária.

obs: utilizando dois métodos distintos para confirmar a estacionariedade

In [ ]:
def teste_dados_estacionarios(series):
    dados_estacionarios_kpss = {}
    dados_estacionarios_adf = {}
    for cd_cvm, serie in series.items():
        print(cd_cvm)
        kpss_stat, p_value, lags, critical_values = kpss(serie)
        
        print(f'Estatística do teste: {kpss_stat:.4f}')
        print(f'p-valor: {p_value:.4f}')
        print('Valores Críticos:')
        for key, value in critical_values.items():
            print(f'{key}: {value:.4f}')
        print('Resultado:')
        if p_value > 0.05:
            #dados_estacionarios_kpss[cd_cvm] = p_value
            dados_estacionarios_kpss[cd_cvm] = 'S'
        else:
            #dados_estacionarios_kpss[cd_cvm] = p_value
            dados_estacionarios_kpss[cd_cvm] = 'N'

        adf_result = adfuller(serie)

        print('ADF Statistic: %f' % adf_result[0])
        print('p-value: %f' % adf_result[1])
        # Optionally, inspect the critical values
        for key, value in adf_result[4].items():
            print(f'Critical Value ({key}): {value:.4f}')

        if adf_result[1] < 0.05:
            #dados_estacionarios_adf[cd_cvm] = adf_result[1]
            dados_estacionarios_adf[cd_cvm] = 'N'
        else:
            dados_estacionarios_adf[cd_cvm] = 'S'

        df_dados_estacionarios_KPSS = pd.DataFrame.from_dict(dados_estacionarios_kpss, orient='index', columns=['KPSS'])
        df_dados_estacionarios_ADF = pd.DataFrame.from_dict(dados_estacionarios_adf, orient='index', columns=['ADF'])
        
    return df_dados_estacionarios_KPSS.join(df_dados_estacionarios_ADF)


In [ ]:
resultado_bruto_estacionariedade = teste_dados_estacionarios(resultado_bruto)
lucro_consolidado_estacionariedade = teste_dados_estacionarios(lucro_consolidado)
despesas_operacionais_estacionariedade = teste_dados_estacionarios(despesas_operacionais)

Armazenar valor do componente de integração para cada série temporal

In [ ]:
def nivel_integracao(series):

    integration_level = {}  # Will store the integration order for each series
    series_corrigido = pd.DataFrame()

    for col in series.columns:
        serie = series[col]
        
        # Apply first-order differencing
        diff1 = serie.diff().dropna()
        adf_result1 = adfuller(diff1)
        
        if adf_result1[1] < 0.05:
            integration_level[col] = 1  # Series became stationary after 1st diff.
            series[col] = diff1
            print(f'{col}: Estacionário depois da primeira diferenciação (I=1)')
        else:
            # Aplicar diferenciação de segunda ordem
            diff2 = diff1.diff().dropna()
            adf_result2 = adfuller(diff2)
            integration_level[col] = 2  # Even after 1st diff, not stationary; use 2nd diff.
            series[col] = diff2
            print(f'{col}: Estaionário depois da segunda diferenciação (I=2) -- p-value: {adf_result2[1]:.4f}')

    print("\nIntegration levels:", integration_level)
    df_integration_level = pd.DataFrame.from_dict(integration_level, orient='index', columns=['Integration Level'])
    return df_integration_level

In [ ]:
nivel_int_resultado_bruto = nivel_integracao(resultado_bruto)
nivel_int_lucro_consolidado = nivel_integracao(lucro_consolidado)
nivel_int_despesas_operacionais = nivel_integracao(despesas_operacionais)

Analise ACF e PACF para séries estacionárias - fluxo para dados estacionarios

Algoritmo

In [ ]:
def analisar_componentes_arima(serie, nlags=10, alpha=0.05):
    """
    Analisa os componentes AR e MA de uma série temporal.

    Parâmetros:
    - serie: pandas Series ou array-like com a série temporal estacionária.
    - nlags: número de lags para avaliar.
    - alpha: nível de significância (default = 5%).

    Retorna:
    - Um dicionário com a conclusão sobre os componentes AR (p) e MA (q).
    """
    
    # Calcular ACF e PACF com intervalos de confiança
    acf_values, acf_interval = acf(serie, nlags=nlags, alpha=alpha)
    # Calcular ACF e PACF com intervalos de confiança
    pacf_values, pacf_interval= pacf(serie, nlags=nlags, alpha=alpha)

    N = len(serie)
    z = norm.ppf(1 - alpha/2)  # por exemplo, 1.96 para 95% de confiança
    
    # Inicializa o array dos intervalos de confiança
    ci_acf = np.zeros((nlags+1, 2))
    ci_pacf = np.zeros((nlags+1, 2))
    
    # Para o lag zero, o ACF é 1 e não calculamos intervalo (ou pode ser [0,0])
    ci_acf[0] = [0, 0]
    ci_pacf[0] = [0, 0]
    
    # Calcula os intervalos para cada lag > 0 usando a fórmula de Bartlett
    for k in range(1, nlags+1):
        var = (1.0 / N) * (1 + 2 * np.sum( acf_values[1:k]**2 ))
        margin = z * np.sqrt(var)
        ci_acf[k] = [-margin, margin]

        # Calcula os intervalos para cada lag > 0 usando a fórmula de Bartlett
    for k in range(1, nlags+1):
        var = (1.0 / N) * (1 + 2 * np.sum( pacf_values[1:k]**2 ))
        margin = z * np.sqrt(var)
        ci_pacf[k] = [-margin, margin]
        
    # Função auxiliar para determinar o maior lag significativo
    def obter_ordem(values, confint):
        ordem = 0
        for lag in range(1, len(values)):  # Ignora lag 0
            if (values[lag] < confint[lag][0]) or (values[lag] > confint[lag][1]):
                ordem = lag
            else:
                # Se o primeiro não significativo aparecer, interrompe
                break
        return ordem

    # Determinar p e q
    #print("Valores ACF:", acf_values)
    #print("Valores PACF:", pacf_values)
    #print("Intervalos de Confiança ACF:", ci_acf)
    #print("Intervalos de Confiança PACF:", ci_pacf)
    p = obter_ordem(pacf_values, ci_pacf)
    q = obter_ordem(acf_values, ci_acf)

    # Construir conclusão
    conclusao = {
        'AR (p)': p,
        'MA (q)': q,
        'Interpretacao': f"O modelo sugerido é ARIMA({p}, d, {q}), onde d é definido pela análise de estacionariedade."
    }

    return p, q, conclusao


Montando variações de ARIMA para cada empresa

In [ ]:
def componentes_arima(serie_temporal, nivel_int_serie):
    resultados = []
    for cd_cvm, series in serie_temporal.items():
        p,q, retorno = analisar_componentes_arima(series)
       # print(f'CD_CVM: {cd_cvm}')
       # print(f'AR(p): {p}, MA(q): {q}')
        serie_temporal_arima_dict = {}
        serie_temporal_arima_dict['CD_CVM'] = cd_cvm
        
        serie_temporal_arima = pd.DataFrame()
        if p == 0 and q == 0:
            #print('Setando p=1 para existir pelo menos um modelo AR')
            p = 1 
            q = 1
        resultados.append({
            'CD_CVM': cd_cvm,
            'AR(p)': p,
            'MA(q)': q
        })

    df_componentes = pd.DataFrame(resultados).set_index('CD_CVM')

    #Juntando componente integrativo com os componentes AR e MA
    df_componentes_arima = pd.concat([nivel_int_serie, df_componentes], axis=1)
    df_componentes_arima.columns = ['I(d)', 'AR(p)', 'MA(q)']
    df_componentes_arima.fillna(0, inplace=True)
    df_componentes_arima['I(d)'] = df_componentes_arima['I(d)'].astype(int)
    print(df_componentes_arima)
    return df_componentes_arima

In [ ]:
def gera_modelo_arima(serie_temporal, df_componentes):
    # Dicionário para armazenar os modelos ajustados para cada CD_CVM
    modelos_arima = {}

    for cd_cvm in df_componentes.index:
        # Split dos dados em treino/validação e teste
        n = len(serie_temporal)
        split_point = int(n * 0.8) 

        p = df_componentes.loc[cd_cvm, 'AR(p)']
        q = df_componentes.loc[cd_cvm, 'MA(q)']
        i = df_componentes.loc[cd_cvm, 'I(d)']
        # Como as séries já são estacionárias, definimos d = 0
        order = (p, i, q)

        serie = serie_temporal[cd_cvm].iloc[:split_point]
        
        try:
            modelo = ARIMA(serie, order=order).fit()
            modelos_arima[cd_cvm] = modelo
            print(f'Modelo ARIMA({p}, {i}, {q}) para CD_CVM {cd_cvm} ajustado com sucesso.')
            print(modelo.summary())
        except Exception as e:
            print(f'Erro ao ajustar modelo para CD_CVM {cd_cvm}: {str(e)}')

    return modelos_arima

In [ ]:
def adicionar_lags(df, lags=4):
    frames = df.copy()  # copia do DataFrame original
    for lag in range(1, lags + 1):
        lagged = df.shift(lag).copy()
        lagged.columns = [f"{col}_lag{lag}" for col in df.columns]
        frames = pd.concat([frames, lagged], axis=1)
    return frames

In [ ]:
resultado_bruto_comp_arima = componentes_arima(resultado_bruto, nivel_int_resultado_bruto)
lucro_consolidado_comp_arima = componentes_arima(lucro_consolidado, nivel_int_lucro_consolidado)
despesas_operacionais_comp_arima = componentes_arima(despesas_operacionais, nivel_int_despesas_operacionais)

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

# Normalizar as variáveis exógenas
# resultado_bruto_normalizado = pd.DataFrame(scaler.fit_transform(resultado_bruto), columns=resultado_bruto.columns, index=resultado_bruto.index)
# lucro_consolidado_normalizado = pd.DataFrame(scaler.fit_transform(lucro_consolidado), columns=resultado_bruto.columns, index=resultado_bruto.index)
# despesas_operacionais_normalizado = pd.DataFrame(scaler.fit_transform(despesas_operacionais), columns=resultado_bruto.columns, index=resultado_bruto.index)

# Normalizar as variáveis exógenas com z-score
resultado_bruto_normalizado = normalize_dataframe(resultado_bruto)
lucro_consolidado_normalizado = normalize_dataframe(lucro_consolidado)
despesas_operacionais_normalizado = normalize_dataframe(despesas_operacionais)

In [ ]:
result_bruto_arima = gera_modelo_arima(resultado_bruto, resultado_bruto_comp_arima)
lucro_consolidado_arima = gera_modelo_arima(lucro_consolidado, lucro_consolidado_comp_arima)
despesas_operacionais_arima = gera_modelo_arima(despesas_operacionais, despesas_operacionais_comp_arima)

In [ ]:
def walk_forward_arima(serie, ordem_arima, n_splits=8):

    # Configurando o TimeSeriesSplit
    # Vamos dividir os dados em 8 janelas de teste. A janela de treino irá crescer a cada passo.
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Listas para armazenar os resultados de cada dobra da validação
    valores_reais = []
    previsoes = []

    print("Iniciando a validação Walk-Forward...")
    # --- 3. O Loop de Validação Walk-Forward ---
    # O tscv.split(serie) gera os índices de treino e teste para cada dobra
    for i, (train_index, test_index) in enumerate(tscv.split(serie)):
        print(f"--- Dobra {i+1}/{n_splits} ---")
        
        # 1. Separar os dados de treino e teste da dobra atual
        treino = serie.iloc[train_index]
        teste = serie.iloc[test_index]
        print(f"Tamanho do Treino: {len(treino)}, Tamanho do Teste: {len(teste)}")
        
        # 2. Treinar o modelo ARIMA APENAS com os dados de treino
        # O modelo é retreinado a cada passo com mais dados, simulando a realidade
        modelo = sm.tsa.ARIMA(treino, order=ordem_arima, )
        modelo_ajustado = modelo.fit()
        
        # 3. Fazer a previsão para o período de teste
        # O número de passos da previsão é o tamanho do conjunto de teste
        previsao = modelo_ajustado.forecast(steps=len(teste))
        
        # 4. Guardar os resultados
        previsoes.extend(previsao)
        valores_reais.extend(teste)

    print("\nValidação Walk-Forward concluída.")

    # --- 4. Avaliação e Visualização dos Resultados ---

    # Calcular a métrica de erro final sobre todos os pontos previstos
    rmse_final = np.sqrt(mean_squared_error(valores_reais, previsoes))
    print(f"\nRMSE Final (out-of-sample): {rmse_final:.4f}")

    # Criar um DataFrame com os resultados para facilitar a visualização
    df_resultados = pd.DataFrame({
        'Real': valores_reais,
        'Previsao': previsoes
    }, index=serie.index[-len(valores_reais):]) # Garante que o índice de datas está correto

    # Plotar os resultados
    plt.figure(figsize=(14, 7))
    plt.plot(serie, label='Série Original Completa', color='gray')
    plt.plot(df_resultados['Real'], label='Valores Reais (Teste)', color='blue', marker='o', linestyle='None')
    plt.plot(df_resultados['Previsao'], label='Previsões Walk-Forward', color='red', linestyle='--')
    plt.title('Validação Walk-Forward - Previsões vs. Real', fontsize=16)
    plt.xlabel('Data')
    plt.ylabel('Valor')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
resultado_bruto[empresa]

In [ ]:
for empresa in resultado_bruto.keys():
    print(f"\nEmpresa {empresa} - Ordem ARIMA: {resultado_bruto_comp_arima.loc[empresa].tolist()}")
    walk_forward_arima(resultado_bruto[empresa], ordem_arima=resultado_bruto_comp_arima.loc[empresa].tolist(), n_splits=8)

In [ ]:
for empresa in lucro_consolidado.keys():
    print(f"\nEmpresa {empresa} - Ordem ARIMA: {lucro_consolidado_comp_arima.loc[empresa].tolist()}")
    walk_forward_arima(lucro_consolidado[empresa], ordem_arima=lucro_consolidado_comp_arima.loc[empresa].tolist(), n_splits=8)

In [ ]:
for empresa in despesas_operacionais.keys():
    print(f"\nEmpresa {empresa} - Ordem ARIMA: {despesas_operacionais_comp_arima.loc[empresa].tolist()}")
    walk_forward_arima(despesas_operacionais[empresa], ordem_arima=despesas_operacionais_comp_arima.loc[empresa].tolist(), n_splits=8)

2 passos de previsão

In [ ]:
from tqdm import tqdm # Biblioteca para adicionar uma barra de progresso (útil para loops longos)


serie = resultado_bruto[21490]


# --- 2. Definição do Modelo e Configuração do Walk-Forward com Horizonte Fixo ---

ordem_arima = (2, 1, 1)
horizonte_previsao = 2 # <-- A MUDANÇA PRINCIPAL: queremos prever 2 passos à frente

# Definir o tamanho do conjunto de treino inicial
# Vamos usar os primeiros 5 anos (20 trimestres) para o primeiro treino
tamanho_treino_inicial = 20

# Listas para armazenar os resultados
valores_reais = []
previsoes_h2 = []
indices_previsao = [] # Guardar os índices de data para o plot

print(f"Iniciando a validação Walk-Forward para horizonte h={horizonte_previsao}...")
# --- 3. O Loop de Validação Walk-Forward Manual ---
# Iteramos a partir do final do treino inicial até o penúltimo ponto possível
for i in tqdm(range(tamanho_treino_inicial, len(serie) - horizonte_previsao + 1)):
    
    # 1. Separar os dados de treino da iteração atual
    treino = serie.iloc[:i]
    
    # 2. Treinar o modelo ARIMA com os dados de treino
    modelo = sm.tsa.ARIMA(treino, order=ordem_arima)
    modelo_ajustado = modelo.fit()
    
    # 3. Fazer a previsão de 'horizonte_previsao' passos à frente
    previsao_passos = modelo_ajustado.forecast(steps=horizonte_previsao)
    
    # 4. Capturar APENAS a previsão do horizonte desejado (a segunda previsão)
    previsao_final = previsao_passos.iloc[horizonte_previsao - 1]
    
    # 5. Capturar o valor real correspondente
    indice_real = i + horizonte_previsao - 1
    valor_real = serie.iloc[indice_real]
    
    # 6. Guardar os resultados
    previsoes_h2.append(previsao_final)
    valores_reais.append(valor_real)
    indices_previsao.append(serie.index[indice_real])

print("\nValidação Walk-Forward concluída.")

# --- 4. Avaliação e Visualização dos Resultados ---

# Calcular a métrica de erro final
rmse_final_h2 = np.sqrt(mean_squared_error(valores_reais, previsoes_h2))
print(f"\nRMSE Final para Horizonte h=2 (out-of-sample): {rmse_final_h2:.4f}")

# Criar um DataFrame com os resultados para facilitar a visualização
df_resultados_h2 = pd.DataFrame({
    'Real': valores_reais,
    'Previsao_h2': previsoes_h2
}, index=indices_previsao)

# Plotar os resultados
plt.figure(figsize=(14, 7))
plt.plot(serie, label='Série Original Completa', color='gray', alpha=0.6)
plt.plot(df_resultados_h2['Real'], label='Valores Reais (Teste, h=2)', color='blue', marker='o', linestyle='None')
plt.plot(df_resultados_h2['Previsao_h2'], label=f'Previsões Walk-Forward (h={horizonte_previsao})', color='red', linestyle='--')
plt.title(f'Validação Walk-Forward - Horizonte de Previsão h={horizonte_previsao}', fontsize=16)
plt.xlabel('Data')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.show()

ARIMAX

Reorganizando indicadores - usados nos modelos

In [ ]:
#meça o tamanho de todas as listas do dict_indicadores
tamanhos = {k: v.shape[0] for k, v in dict_indicadores.items()}
tamanhos

Criando lags

In [ ]:
dict_indicadores[empresa].columns

In [ ]:
#adicionar lags aos indicadores
df_lags_indicadores = {}
dict_indicadores_select = dict_indicadores.copy()

for empresa in dict_indicadores_select.keys(): 
    #Removendo indicadores que serão previstos nos modelos
    dict_indicadores_select[empresa].drop(columns=['3.01', '3.02', '3.11'], inplace=True)
    df_lags_indicadores[empresa] = adicionar_lags(dict_indicadores_select[empresa], lags=4)
    df_lags_indicadores[empresa] = df_lags_indicadores[empresa].iloc[8:]
    #renomear index
    print(df_lags_indicadores[empresa].columns)
    #normalizar os valores
    df_lags_indicadores[empresa] = (df_lags_indicadores[empresa] - df_lags_indicadores[empresa].mean()) / df_lags_indicadores[empresa].std()

#adicionar lags às entradas
df_lags_entradas = adicionar_lags(indices_estac, lags=8)
#df_lags_entradas = df_lags_entradas.iloc[4:]
#df_lags_entradas.set_index('data', inplace=True)


exog_vars_estac_norm_lags = {}
for empresa in dict_indicadores.keys(): 
    #Removendo lags 0 dos indicadores, uma vez que não teremos estes valores
    dict_indicadores_select[empresa] = dict_indicadores_select[empresa].drop(columns=['3.03', '3.04', '3.05', '3.06', '3.07', '3.08', '3.09'])
    exog_vars_estac_norm_lags[empresa] = pd.concat([dict_indicadores_select[empresa], df_lags_entradas], axis=1)
#antes de alimentar os modelos, dependendo da saída do modelo, tirar o indicador referente

In [ ]:
exog_vars_estac_norm_lags[2437].columns

Escolher as variáveis a serem utilizadas

In [ ]:
input_arimax = {}
for empresa in dict_indicadores.keys(): 
    for indicador in ['3.01', '3.02', '3.11']:
            input_arimax[(empresa, indicador)] = exog_vars_estac_norm_lags[empresa].loc[dict_indicadores[empresa].index]

In [ ]:
exog_vars_estac_norm_lags[empresa][['ibov', 'ibov_lag1']]

Agrupando exog_vars - sem lags

In [ ]:
#normalizar dict_indicadores_estac
dict_indicadores_estac_norm = {}
for empresa in dict_indicadores_estac.keys():
    dict_indicadores_estac_norm[empresa] = (dict_indicadores_estac[empresa] - dict_indicadores_estac[empresa].mean()) / dict_indicadores_estac[empresa].std()

In [ ]:
exog_vars_estac = {}
for empresa in dict_indicadores.keys(): 
    exog_vars_estac[empresa] = pd.concat([dict_indicadores_estac_norm[empresa].drop(columns=['3.01', '3.02', '3.11']), indices_norm_estac], axis=1)
#antes de alimentar os modelos, dependendo da saída do modelo, tirar o indicador referente

In [ ]:
exog_vars_estac[2437].shape

VIF

In [ ]:
def calcular_vif_iterativo(df, thresh=5.0, max_iter=100, verbose=True):
    """
    Calcula VIF iterativamente removendo a variável com maior VIF até que todos fiquem <= thresh.
    Retorna:
      - df_filtrado: DataFrame final após remoções
      - vif_final: Series com VIF das variáveis remanescentes
      - removidas: DataFrame com ordem de remoção e VIF no momento da remoção
    """
    # Copiar e manter apenas colunas numéricas
    df_work = df.select_dtypes(include=[np.number]).copy()

    # Remover colunas constantes ou quase-constantes
    const_cols = [c for c in df_work.columns if df_work[c].nunique(dropna=True) <= 1]
    if const_cols and verbose:
        print(f"Removendo colunas constantes: {const_cols}")
    df_work.drop(columns=const_cols, inplace=True, errors='ignore')

    # Limpeza de dados: remover linhas com NaN/Inf
    df_work = df_work.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how='any')

    removidas = []
    it = 0
    while it < max_iter and df_work.shape[1] > 1:
        it += 1
        X = sm.add_constant(df_work, has_constant='add')
        vifs = []
        for i, col in enumerate(X.columns):
            try:
                val = variance_inflation_factor(X.values, i)
            except Exception:
                val = np.inf
            vifs.append(val)
        vif = pd.Series(vifs, index=X.columns)

        # descartar a constante
        if 'const' in vif.index:
            vif = vif.drop('const', errors='ignore')

        # Se todas as variáveis estão abaixo do limiar, parar
        max_v = vif.max()
        if pd.isna(max_v) or max_v <= thresh:
            if verbose:
                print("\nTodos os VIFs estão abaixo do limite.")
            break

        # Remover a variável com maior VIF (inclusive se for inf)
        col_remover = vif.idxmax()
        if verbose:
            print(f"Removendo '{col_remover}' com VIF de {max_v:.2f}")
        removidas.append({'variavel': col_remover, 'vif': float(max_v), 'iter': it})
        df_work = df_work.drop(columns=[col_remover])

        # Se sobrar 1 coluna, não há por que continuar
        if df_work.shape[1] <= 1:
            break

    # VIF final das variáveis remanescentes
    if df_work.shape[1] >= 1:
        Xf = sm.add_constant(df_work, has_constant='add')
        vifs_final = []
        for i, col in enumerate(Xf.columns):
            try:
                val = variance_inflation_factor(Xf.values, i)
            except Exception:
                val = np.inf
            vifs_final.append(val)
        vif_final = pd.Series(vifs_final, index=Xf.columns).drop('const', errors='ignore').sort_values(ascending=False)
    else:
        vif_final = pd.Series(dtype=float)

    removidas_df = pd.DataFrame(removidas)
    if verbose:
        print("\nDataFrame final após remoção por VIF:")
        display(df_work.head())
        print("\nValores finais de VIF:")
        display(vif_final)

    return df_work, vif_final, removidas_df




# ...existing code...

In [ ]:
# Exemplo: aplicar em todos os dataframes de exog_vars_estac
vif_filtrados = {}
vif_finais = {}
vif_remocoes = {}

for empresa, df_exog in exog_vars_estac.items():
    print(f"\n=== Empresa {empresa} ===")
    df_filtrado, vif_final, removidas = calcular_vif_iterativo(df_exog, thresh=5.0, verbose=True)
    vif_filtrados[empresa] = df_filtrado
    vif_finais[empresa] = vif_final
    vif_remocoes[empresa] = removidas

A partir deste momentos as entradas estão selecionadas

In [ ]:
vif_filtrados[empresa].columns

Correlação Cruzada pós-VIF

In [ ]:
# ...existing code...
# Calcular a Função de Correlação Cruzada (CCF) entre índices (VIF) e apenas os indicadores alvo
ccf_dict = {}
alvo_indicadores = ['3.01', '3.02', '3.11']

# Número de observações para o cálculo do intervalo de confiança (aprox.)
n_obs = len(indices_norm_estac)
conf_interval = 1.96 / np.sqrt(n_obs)

for empresa, df in dict_indicadores_estac.items():
    # Indicadores alvo disponíveis neste df
    cols_indicadores = [c for c in alvo_indicadores if c in df.columns]
    if not cols_indicadores:
        continue

    # # Índices aprovados pelo VIF que existem em indices_norm_estac
    # cols_indices = [c for c in vif_finais.get(empresa, pd.Series(dtype=float)).index
    #                 if c in indices_norm_estac.columns]
    # if not cols_indices:
    #     continue

    for indice in vif_filtrados[empresa].columns:
        serie_x = vif_filtrados[empresa][indice].dropna()
        for col in cols_indicadores:
            serie_y = df[col].dropna()

            # Alinhar pelas datas em comum
            aligned = pd.concat([serie_x, serie_y], axis=1, join='inner').dropna()
            if len(aligned) < 3:
                continue

            corr = ccf(aligned.iloc[:, 0], aligned.iloc[:, 1], adjusted=False)
            lags = np.arange(len(corr))
            ccf_dict[(empresa, indice, col)] = corr

In [ ]:
# ...existing code...
def plot_ccf(lags, corr, indice, col, empresa=None):
    """
    Plota a CCF com linhas de confiança, eixos e legendas padronizados.
    - lags: array de lags
    - corr: array de correlações (mesmo tamanho de lags)
    - indice: nome da variável X
    - indicador: nome da variável Y
    - empresa: identificador opcional
    - n_obs: nº de observações para IC; se None usa len(corr) como aproximação
    - alpha: nível de significância (default 5%)
    """
    fig, ax =plt.subplots(figsize=(15, 6))
    plt.stem(lags, corr, use_line_collection=True)
    plt.title(f'CCF entre {indice} e {col} - Empresa {empresa}')
    confidence_interval = 1.96/np.sqrt(len(corr))
    ax.axhline(-confidence_interval, color='red', label='5% intervalo de confiança')
    ax.axhline(confidence_interval, color='red')
    ax.axvline(x = 0, color = 'black', lw = 1)
    ax.axhline(y = 0, color = 'black', lw = 1)
    ax.axhline(y = np.max(corr), color = 'blue', lw = 1,
            linestyle='--', label = 'Maior correlação +/-')
    ax.axhline(y = np.min(corr), color = 'blue', lw = 1,
            linestyle='--')
    ax.set(ylim = [-1, 1])
    # if y_label != x_label:
    #     ax.set_title('Correlação Cruzada ' + x_label + ' e ' + y_label, weight='bold', fontsize = 15)
    # if y_label == x_label:
    #     ax.set_title('Autocorrelação ' + x_label, weight='bold', fontsize = 15)
    ax.set_ylabel('Coeficientes de Correlação', weight='bold',
    fontsize = 12)
    #determina os pontos marcados no eixo X
    plt.xticks(range(min(lags), max(lags+1), 3))
    ax.set_xlabel('Lags', weight='bold', fontsize = 12)
    plt.legend()
# ...existing code...

In [ ]:
# ...existing code...
plot_ccf(lags, corr, indice, col, empresa)

Mostrar maiores correlações cruzadas

In [ ]:
# Desempacotar ccf_dict em um DataFrame
records = []
for (empresa, indice_x, indicador), ccf_arr in ccf_dict.items():
    for lag, corr_val in enumerate(ccf_arr):
        records.append({
            'Indice': indice_x,
            'Empresa': empresa,
            'Indicador': indicador,
            'Lag': lag,          # statsmodels.ccf retorna lags >= 0
            'CCF': corr_val,
            'Abs_CCF': abs(corr_val)
        })

df_ccf = pd.DataFrame(records)

# Opcional: limitar por lag máximo (ex.: até 8)
lag_max = 8
df_ccf = df_ccf[df_ccf['Lag'] <= lag_max]

# Marcar significância pelo intervalo de confiança já calculado (conf_interval)
df_ccf['Significativo'] = df_ccf['Abs_CCF'] > conf_interval

# 1) Top N lags significativos em geral
topN = 10
top_lags = (df_ccf[df_ccf['Significativo']]
            .sort_values('Abs_CCF', ascending=False)
            .head(topN))
display(top_lags)

# 2) Ranking por par (Indice, Empresa, Indicador): melhor lag por par
idx_max = (df_ccf[df_ccf['Significativo']]
           .groupby(['Empresa', 'Indice', 'Indicador'])['Abs_CCF']
           .idxmax())
ranked_pairs = (df_ccf.loc[idx_max]
                .sort_values('Abs_CCF', ascending=False)
                .reset_index(drop=True))
display(ranked_pairs.head(topN))

Monta o ranking das correlações (elimina o lag zero dos demais indicadores)

In [ ]:
# ...existing code...
# Desempacotar ccf_dict em um DataFrame
records = []
for (empresa, indice_x, indicador), ccf_arr in ccf_dict.items():
    for lag, corr_val in enumerate(ccf_arr):
        records.append({
            'Indice': indice_x,
            'Empresa': empresa,
            'Indicador': indicador,
            'Lag': lag,          # statsmodels.ccf retorna lags >= 0
            'CCF': corr_val,
            'Abs_CCF': abs(corr_val)
        })

df_ccf = pd.DataFrame(records)

# Limitar por lag máximo (ex.: até 8)
lag_max = 4
df_ccf = df_ccf[df_ccf['Lag'] <= lag_max].copy()

# Marcar significância
df_ccf['Significativo'] = df_ccf['Abs_CCF'] > conf_interval

# Remover apenas Lag==0 dos indicadores 3.0X (3.01..3.09)
mask_30x_lag0 = (df_ccf['Lag'] == 0) & df_ccf['Indice'].astype(str).str.match(r'^3\.0\d$')
df_ccf_filt = df_ccf[~mask_30x_lag0].copy()

# Top N lags significativos (pós-filtro)
topN = 5
top_lags = (df_ccf_filt[df_ccf_filt['Significativo']]
            .sort_values('Abs_CCF', ascending=False)
            .head(topN))
display(top_lags)

# Melhor lag por par (pós-filtro)
subset_sig = df_ccf_filt[df_ccf_filt['Significativo']]
if not subset_sig.empty:
    idx_max = (subset_sig
               .groupby(['Empresa', 'Indice', 'Indicador'])['Abs_CCF']
               .idxmax())
    ranked_pairs = (subset_sig.loc[idx_max]
                    .sort_values('Abs_CCF', ascending=False)
                    .reset_index(drop=True))
else:
    ranked_pairs = pd.DataFrame(columns=['Empresa','Indice','Indicador','Lag','CCF','Abs_CCF','Significativo'])

display(ranked_pairs.head(topN))
# ...existing code...

In [ ]:
ranked_pairs.to_csv('ranked_ccf_pairs.csv', index=False)

In [ ]:
ranked_pairs.query('Empresa == 2437 and Indicador == "3.11"').sort_values(by='Abs_CCF', ascending=False)

In [ ]:
# Plotar Top 5 por (Empresa, Indicador) com base em ranked_pairs
top_n = 5
alvo_indicadores = ['3.01', '3.02', '3.11']  # opcional: filtre apenas os indicadores-alvo

rp = ranked_pairs.copy()
if 'Indicador' in rp.columns and alvo_indicadores:
    rp = rp[rp['Indicador'].isin(alvo_indicadores)]

for (empresa, indicador), df_group in rp.groupby(['Empresa', 'Indicador']):
    df_top = df_group.sort_values('Abs_CCF', ascending=False).head(top_n)
    for _, row in df_top.iterrows():
        indice = row['Indice']
        key = (empresa, indice, indicador)
        corr = ccf_dict.get(key)
        if corr is None:
            # chave inexistente no dicionário (pode ocorrer por nomes/filtragens)
            continue
        lags = np.arange(len(corr))
        plot_ccf(lags, corr, indice=indice, col=indicador, empresa=empresa)

Montando grupo de variáveis exógenas

In [ ]:
# ...existing code...
# Monta DataFrames filtrados de exog_vars_estac_norm_lags usando ranked_pairs (indicador + lag)
# Agora armazena por empresa e por indicador: exog_vars_lags_filtrado[empresa][indicador] = DataFrame

exog_vars_lags_filtrado = {}

for emp in ranked_pairs['Empresa'].unique():
    exog_vars_lags_filtrado[emp] = {}
    df_emp = ranked_pairs[ranked_pairs['Empresa'] == emp]
    for indicador in df_emp['Indicador'].unique():
        df_ind = df_emp[df_emp['Indicador'] == indicador]
        # Para cada linha, monta o nome da coluna: indicador_lagN
        col_names = [f"{row['Indice']}_lag{row['Lag']}" for _, row in df_ind.iterrows()]
        # Filtra apenas colunas existentes no DataFrame de lags
        cols_exist = [c for c in col_names if c in exog_vars_estac_norm_lags[emp].columns]
        # Monta o DataFrame filtrado para este indicador
        exog_vars_lags_filtrado[emp][indicador] = exog_vars_estac_norm_lags[emp][cols_exist].copy()
        print(f"Empresa {emp} - Indicador {indicador}: {len(cols_exist)} colunas -> {cols_exist}")

# Exemplo de visualização
# Para empresa 2437 e indicador '3.11':
display(exog_vars_lags_filtrado[2437]['3.11'].head())
#

In [ ]:
import re

top_n = 5
alvo_indicadores = ['3.01', '3.02', '3.11']  # opcional

# Garantir que estamos aplicando a regra do Lag==0 para 3.0X também aqui
rp = ranked_pairs.copy()
if 'Indicador' in rp.columns and alvo_indicadores:
    rp = rp[rp['Indicador'].isin(alvo_indicadores)]

mask_lag0_30x = (rp['Lag'] == 0) & rp['Indicador'].astype(str).str.match(r'^3\.0\d{2}$')
rp = rp[~mask_lag0_30x]

# Top N por (Empresa, Indicador)
rp_top = (rp.sort_values('Abs_CCF', ascending=False)
            .groupby(['Empresa', 'Indicador'])
            .head(top_n))

# Lista de índices por empresa
indices_por_empresa = (rp_top.groupby('Empresa')['Indice']
                        .agg(lambda s: sorted(pd.unique(s)))
                        .to_dict())

# Dict de DataFrames filtrados por empresa (mantém apenas colunas existentes)
indices_norm_estac_filtrado = {}
for emp, cols in indices_por_empresa.items():
    cols_exist = [c for c in cols if c in indices_norm_estac.columns]
    indices_norm_estac_filtrado[emp] = indices_norm_estac[cols_exist].copy()
    print(f'Empresa {emp}: {len(cols_exist)} colunas -> {cols_exist}')

# DataFrame único com a união das colunas selecionadas em todas as empresas
cols_union = sorted({c for cols in indices_por_empresa.values() for c in cols if c in indices_norm_estac.columns})
indices_norm_estac_union = indices_norm_estac[cols_union].copy()

# Visualização rápida
display(indices_norm_estac_union.head())

Teste modelo Arimax

In [ ]:
def gera_modelo_arimax(serie_temporal, df_componentes, exog_dict):
    modelos_arimax = {}

    for cd_cvm in df_componentes.index:
        series = serie_temporal.copy()[cd_cvm][4:]
        n = len(series)
        split_point = int(n * 0.8)  # 80% dos dados para treino
        print(f"CD_CVM: {cd_cvm}, Length: {n}, Split Point: {split_point}")
        p = df_componentes.loc[cd_cvm, 'AR(p)']
        q = df_componentes.loc[cd_cvm, 'MA(q)']
        i = df_componentes.loc[cd_cvm, 'I(d)']
        order = (p, i, q)

        serie = series.iloc[:split_point]

        exog_vars = exog_dict[cd_cvm].iloc[:split_point]

        try:
            print(f"Serie: {serie.shape}, Exog: {exog_vars.shape}, Order: {order}")
            modelo = ARIMA(serie, order=order, exog=exog_vars[:split_point]).fit()
            modelos_arimax[cd_cvm] = modelo
            #print(f'Modelo ARIMAX({p}, {i}, {q}) para CD_CVM {cd_cvm} ajustado com sucesso.')
            #print(modelo.summary())
        except Exception as e:
            print(f'Erro ao ajustar modelo ARIMAX para CD_CVM {cd_cvm}: {str(e)}')

    return modelos_arimax

In [ ]:
resultado_bruto_norm = {}
for empresa in resultado_bruto.columns:
    resultado_bruto_norm[empresa] = (resultado_bruto[empresa] - resultado_bruto[empresa].mean()) / resultado_bruto[empresa].std()

In [ ]:
#prepara input para o modelo ARIMAX
for empresa in exog_vars_estac.keys():
    #remover colunas com NaN
    
    zeros_por_coluna = (exog_vars_estac[empresa] == 0).sum()
    display(zeros_por_coluna[zeros_por_coluna > 0])


In [ ]:
input = {}
for empresa in exog_vars_estac.keys():
    input[empresa] = exog_vars_estac[empresa]['incc'][8:]

In [ ]:
input = {}
for empresa in exog_vars_estac.keys():
    input[empresa] = exog_vars_lags_filtrado[emp][indicador].iloc[4:]

In [ ]:
input[2437]

In [ ]:
modelo_arimax_resultado_bruto = gera_modelo_arimax(resultado_bruto_norm, resultado_bruto_comp_arima, input)

In [ ]:
arimax_forecast_result = aplica_modelo_forecast(modelo_arimax_resultado_bruto, resultado_bruto_normalizado,input)

In [ ]:
df_combined_result_final['ARIMAX RMSE'] = arimax_forecast_result['RMSE']
df_combined_result_final['ARIMAX MAE'] = arimax_forecast_result['MAE']
df_combined_result_final['ARIMAX RMSE Improvement (%)'] = ((df_combined_result_final['Naive RMSE'] - df_combined_result_final['ARIMAX RMSE']) / df_combined_result_final['Naive RMSE']) * 100
df_combined_result_final.sort_values(by='ARIMAX RMSE Improvement (%)', ascending=False)

In [ ]:
# Cria uma lista de dicionários com os valores de AIC e BIC para cada cd_cvm -> RESULTADO BRUTO
performance = []
for cd_cvm, modelo in result_bruto_arima.items():
    performance.append({
        'CD_CVM': cd_cvm,
        'AIC': modelo.aic,
        'BIC': modelo.bic
    })

# Cria o DataFrame e define CD_CVM como índice
df_performance_resul_bruto = pd.DataFrame(performance).set_index('CD_CVM')

# Ordena o DataFrame pelo AIC (menor é melhor)
df_performance_resul_bruto = df_performance_resul_bruto.sort_values(by='AIC')
df_combined_result = df_performance_resul_bruto.join(arimax_forecast_result)
print(df_combined_result.sort_values(by='CD_CVM'))